In [2]:
# Basic imports
import scipy, pandas as pd, numpy as np
from matplotlib import pyplot as plt

In [3]:
INCLUDED = ['Coastal Flood', 'Heavy Rain', 'Hurricane (Typhoon)', 'Drought', 'Lakeshore Flood', 'Flash Flood', 'Flood', 'Thunderstorm Wind', 'Tropical Depression', 'Tropical Storm', 'Tsunami', 'Storm Surge/Tide', 'Tornado']

In [4]:
tdet = pd.read_csv("NCEI Data/StormEvents_details-ftp_v1.0_d2006_c20220425.csv")

tloc = pd.read_csv("NCEI Data/StormEvents_locations-ftp_v1.0_d2006_c20220425.csv")

test = pd.merge(tdet, tloc, on="EVENT_ID", how='outer')

C:\Users\ew0045\AppData\Local\Temp\ipykernel_11468\3960564854.py:1: DtypeWarning: Columns (29,34,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  tdet = pd.read_csv("NCEI Data/StormEvents_details-ftp_v1.0_d2006_c20220425.csv")


In [5]:
tdet.dtypes

BEGIN_YEARMONTH         int64
BEGIN_DAY               int64
BEGIN_TIME              int64
END_YEARMONTH           int64
END_DAY                 int64
END_TIME                int64
EPISODE_ID              int64
EVENT_ID                int64
STATE                  object
STATE_FIPS              int64
YEAR                    int64
MONTH_NAME             object
EVENT_TYPE             object
CZ_TYPE                object
CZ_FIPS                 int64
CZ_NAME                object
WFO                    object
BEGIN_DATE_TIME        object
CZ_TIMEZONE            object
END_DATE_TIME          object
INJURIES_DIRECT         int64
INJURIES_INDIRECT       int64
DEATHS_DIRECT           int64
DEATHS_INDIRECT         int64
DAMAGE_PROPERTY        object
DAMAGE_CROPS           object
SOURCE                 object
MAGNITUDE             float64
MAGNITUDE_TYPE         object
FLOOD_CAUSE            object
CATEGORY              float64
TOR_F_SCALE            object
TOR_LENGTH            float64
TOR_WIDTH 

In [6]:
def proc(detFrame, locFrame, included):
    mid = pd.merge(detFrame, locFrame, on="EVENT_ID", how='left')
    # Specify the columns you want to keep
    columns_to_keep = ['EVENT_ID', 'BEGIN_YEARMONTH', 'BEGIN_DAY', 'END_YEARMONTH', 'END_DAY','STATE_FIPS', 'CZ_TYPE', 'CZ_FIPS', 'EVENT_TYPE', 'LATITUDE', 'LONGITUDE']

    # Drop all columns except the specified columns
    columns_to_drop = mid.columns.difference(columns_to_keep)
    dropped = mid.drop(columns=columns_to_drop)

    events = dropped[dropped['EVENT_TYPE'].isin(included)]

    events['year_month_day_beg'] = events['BEGIN_YEARMONTH'].astype(str) + events['BEGIN_DAY'].astype(str).str.zfill(2)

    events['beg_datetime'] = pd.to_datetime(events['year_month_day_beg'], format='%Y%m%d')

    events['year_month_day_end'] = events['END_YEARMONTH'].astype(str) + events['END_DAY'].astype(str).str.zfill(2)

    events['end_datetime'] = pd.to_datetime(events['year_month_day_end'], format='%Y%m%d')

    events.drop('year_month_day_beg', axis=1, inplace=True)

    events.drop('year_month_day_end', axis=1, inplace=True)

    events['days'] = (events['end_datetime'] - events['beg_datetime']).dt.days + 1

    cOrder = ['EVENT_ID', 'STATE_FIPS', 'CZ_TYPE', 'CZ_FIPS', 'EVENT_TYPE', 'BEGIN_YEARMONTH', 'BEGIN_DAY', 'END_YEARMONTH', 'END_DAY', 'LATITUDE', 'LONGITUDE', 'beg_datetime', 'end_datetime', 'days']

    reordered = events[cOrder]

    return reordered

In [7]:
import os

detFiles = {}

# Directory containing your files
directory = "NCEI Data/"

# Iterate through files in the directory
for filename in os.listdir(directory):
    # Check if filename starts with the pattern "StormEvents_details-ftp_v1.0_d"
    if filename.startswith("StormEvents_details-ftp_v1.0_d"):
        # Extract the year from the filename
        year = filename[30:34]  # Assuming the year is always at positions 36 to 39
        # Process the rest of the filename
        rest_of_filename = filename[40:]  # Adjust this index based on your actual filename structure
        # Now you can use year and rest_of_filename to import your file or perform any other operations
        # print("Filename:", filename)
        # print("Year:", year)
        # print("Rest of filename:", rest_of_filename)
        detFiles[year] = directory + filename

In [89]:
file = pd.read_csv("ZonetoFIPS.txt", names=["STATE", "ZONE", "CWA", "NAME", "STATE_ZONE", "COUNTY", "FIPS", "TIME_ZONE", "FE_AREA", "LAT", "LON"], sep='|')

In [99]:
file.loc[(file['ZONE'] == 2) & (file['STATE'] == 'CO')]

,STATE,ZONE,CWA,NAME,STATE_ZONE,COUNTY,FIPS,TIME_ZONE,FE_AREA,LAT,LON
1547,CO,2,GJT,Central Yampa River Basin,CO002,Moffat,8081,M,nw,40.4046,-107.9134
1548,CO,2,GJT,Central Yampa River Basin,CO002,Rio Blanco,8103,M,nw,40.4046,-107.9134
1549,CO,2,GJT,Central Yampa River Basin,CO002,Routt,8107,M,nw,40.4046,-107.9134


: 

In [10]:
locFiles = {}

for filename in os.listdir(directory):
    # Check if filename starts with the pattern "StormEvents_details-ftp_v1.0_d"
    if filename.startswith("StormEvents_locations-ftp_v1.0_d"):
        # Extract the year from the filename
        year = filename[32:36]  # Assuming the year is always at positions 36 to 39
        # Now you can use year and rest_of_filename to import your file or perform any other operations
        print("Filename:", filename)
        print("Year:", year)
        locFiles[year] = directory + filename

Filename: StormEvents_locations-ftp_v1.0_d2006_c20220425.csv
Year: 2006
Filename: StormEvents_locations-ftp_v1.0_d2007_c20240216.csv
Year: 2007
Filename: StormEvents_locations-ftp_v1.0_d2008_c20220425.csv
Year: 2008
Filename: StormEvents_locations-ftp_v1.0_d2009_c20231116.csv
Year: 2009
Filename: StormEvents_locations-ftp_v1.0_d2010_c20220425.csv
Year: 2010
Filename: StormEvents_locations-ftp_v1.0_d2011_c20230417.csv
Year: 2011
Filename: StormEvents_locations-ftp_v1.0_d2012_c20221216.csv
Year: 2012
Filename: StormEvents_locations-ftp_v1.0_d2013_c20230118.csv
Year: 2013
Filename: StormEvents_locations-ftp_v1.0_d2014_c20231116.csv
Year: 2014
Filename: StormEvents_locations-ftp_v1.0_d2015_c20220425.csv
Year: 2015
Filename: StormEvents_locations-ftp_v1.0_d2016_c20220719.csv
Year: 2016
Filename: StormEvents_locations-ftp_v1.0_d2017_c20230317.csv
Year: 2017
Filename: StormEvents_locations-ftp_v1.0_d2018_c20230616.csv
Year: 2018
Filename: StormEvents_locations-ftp_v1.0_d2019_c20240117.csv
Yea

In [11]:
print(detFiles)

{'2006': 'NCEI Data/StormEvents_details-ftp_v1.0_d2006_c20220425.csv', '2007': 'NCEI Data/StormEvents_details-ftp_v1.0_d2007_c20240216.csv', '2008': 'NCEI Data/StormEvents_details-ftp_v1.0_d2008_c20220425.csv', '2009': 'NCEI Data/StormEvents_details-ftp_v1.0_d2009_c20231116.csv', '2010': 'NCEI Data/StormEvents_details-ftp_v1.0_d2010_c20220425.csv', '2011': 'NCEI Data/StormEvents_details-ftp_v1.0_d2011_c20230417.csv', '2012': 'NCEI Data/StormEvents_details-ftp_v1.0_d2012_c20221216.csv', '2013': 'NCEI Data/StormEvents_details-ftp_v1.0_d2013_c20230118.csv', '2014': 'NCEI Data/StormEvents_details-ftp_v1.0_d2014_c20231116.csv', '2015': 'NCEI Data/StormEvents_details-ftp_v1.0_d2015_c20220425.csv', '2016': 'NCEI Data/StormEvents_details-ftp_v1.0_d2016_c20220719.csv', '2017': 'NCEI Data/StormEvents_details-ftp_v1.0_d2017_c20230317.csv', '2018': 'NCEI Data/StormEvents_details-ftp_v1.0_d2018_c20230616.csv', '2019': 'NCEI Data/StormEvents_details-ftp_v1.0_d2019_c20240117.csv', '2020': 'NCEI Data/

In [12]:
years = range(2006, 2023)
columnNames = ['EVENT_ID', 'STATE_FIPS', 'CZ_TYPE', 'CZ_FIPS', 'EVENT_TYPE', 'BEGIN_YEARMONTH', 'BEGIN_DAY', 'END_YEARMONTH', 'END_DAY', 'LATITUDE', 'LONGITUDE', 'beg_datetime', 'end_datetime', 'days']

for year in years:
    detFrame = pd.read_csv(detFiles[str(year)])
    locFrame = pd.read_csv(locFiles[str(year)])
    result = proc(detFrame, locFrame, INCLUDED)

    np.savetxt("stormproc" + str(year) + ".csv", result, fmt='%s', delimiter=', ', header=', '.join(columnNames))


C:\Users\ew0045\AppData\Local\Temp\ipykernel_11468\2643656432.py:5: DtypeWarning: Columns (29,34,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  detFrame = pd.read_csv(detFiles[str(year)])
C:\Users\ew0045\AppData\Local\Temp\ipykernel_11468\2427541654.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events['year_month_day_beg'] = events['BEGIN_YEARMONTH'].astype(str) + events['BEGIN_DAY'].astype(str).str.zfill(2)
C:\Users\ew0045\AppData\Local\Temp\ipykernel_11468\2427541654.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

In [13]:
detFrame = pd.read_csv("NCEI Data/StormEvents_details-ftp_v1.0_d2022_c20240216.csv")
locFrame = pd.read_csv("NCEI Data/StormEvents_locations-ftp_v1.0_d2022_c20240216.csv")